In [ ]:
import pandas as pd

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional


In [ ]:
X_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_train.csv", header=None)
X_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_test.csv", header=None)
y_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_train.csv", header=None)
y_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_test.csv", header=None)



print("X train ",X_train.shape)
print("X test",X_test.shape)
print("Y_Train", y_train.shape)
print("Y_test", y_test.shape)

In [ ]:
import keras
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
)

def compile_model(model):

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

def fit_model(model, epochs, X_train, y_train,
              X_test, y_test, ):

    history = model.fit(X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=epochs,
                batch_size=64,
                verbose=1)

    return history


def get_classification_metrics(y_test: pd.DataFrame, y_pred: pd.DataFrame) -> dict[str, str]:
    """
    Returns the classification report as a dictionary.
    """
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred),
        "fb_score": fbeta_score(y_test, y_pred, beta=0.2),
    }


In [ ]:
def create_model():
    """
    Create and return the model I wanna try on.
    Which is a nueral network with one hidden layer and a dropout layer.
    """
    model = Sequential()

    model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(2000, 1))))
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    return model

model = create_model()

compile_model(model)
fit_model(model, 50, X_train, y_train, X_test, y_test)
